In [ ]:
import ee
import geemap

Map = geemap.Map()

def NDVI_V1(img):
 nir = img.select("B8")
 red = img.select("B4")
 ndvi = nir.subtract(red).divide(nir.add(red))
 return ndvi

def NDVI_V2(img):
 nir = img.select("B8")
 red = img.select("B4")
 ndvi = img.expression(
   "(B5 - B4)/(B5 + B4)",
   {
     "B5": nir,
     "B4": red
   }
 )
 return ndvi

def NDVI_V3(img):
 ndvi = img.normalizedDifference(["B8","B4"])
 return ndvi

#sentinel2 and roi
s2_col = ee.ImageCollection("COPERNICUS/S2")
roi = ee.Geometry.Point([117.0703125,38.09133660751176])
img = ee.Image(s2_col.filterBounds(roi) \
                       .filterDate("2017-01-01", "2017-09-24") \
                       .first())
ndvi1 = NDVI_V1(img)
ndvi2 = NDVI_V2(img)
ndvi3 = NDVI_V3(img)
visParam = {
 'min': -0.2,
 'max': 0.8,
 'palette': 'FFFFFF, CE7E45, DF923D, F1B555, FCD163, 99B718, 74A901, 66A000, 529400,' + \
   '3E8601, 207401, 056201, 004C00, 023B01, 012E01, 011D01, 011301'
}
Map.addLayer(img, {'bands':["B8",  "B4",  "B3"], 'max':3048}, "raw_img")
Map.addLayer(ndvi1, visParam, "ndvi_1")
Map.addLayer(ndvi2, visParam, "ndvi_2")
Map.addLayer(ndvi3, visParam, "ndvi_3")
Map.centerObject(roi, 9)

#show charts
ndvi_list = s2_col.filterDate("2017-01-01", "2017-09-24")

def func_fui(image):
   ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
   return image.addBands(ndvi) \
   .map(func_fui)

chart1 = ui.Chart.image.series({
 'imageCollection': ndvi_list.select('NDVI'),
 'region': roi,
 'reducer': ee.Reducer.mean(),
 'scale': 30
}).setOptions({'title': 'NDVI IMAGE SERIES'})
print(chart1)

chart2 = ui.Chart.image.doySeries({
 'imageCollection': ndvi_list.select('NDVI'),
 'region':roi,
 'regionReducer': ee.Reducer.mean(),
 'scale':30
}).setOptions({'title': "ROI NDVI EACH DAY SERIES"})
print(chart2)
Map